# Demo for XYZ Spaces for Python

This notebook shows a demo of xyzspaces.

In [ ]:
# This is only to show what to integrate better into Jupyter in the future
# (not covered further in this notebook).

from IPython.display import IFrame

url = "https://xyz.here.com/viewer/"
project_id = "5c54716d-f900-4b89-80ac-b21518e94b30"
IFrame(src=f"{url}?project_id={project_id}", width="100%", height=400)

In [ ]:
import os

import requests
import warnings
from sidecar import Sidecar
from ipyleaflet import Map, GeoJSON, FullScreenControl

from xyzspaces.apis import HubApi
from xyzspaces.datasets import get_countries_data
import xyzspaces

<div class="alert alert-block alert-warning">
<b>Warning:</b> Before running below cells please make sure you have XYZ Token to interact with xyzspaces. 
                Please see README.md in notebooks folder for more info on XYZ_TOKEN
</div>

In [ ]:
# Make a XYZ object
try:
    xyz_token = os.environ["XYZ_TOKEN"]
except KeyError:
    xyz_token = "MY-FANCY-XYZ-TOKEN"
    if xyz_token == "MY-FANCY-XYZ-TOKEN":
        warnings.warn(
            "Please either set XYZ_TOKEN to env variable or "
            "just assign value of your actual token to variable xyz_token above."
        )
xyz = xyzspaces.XYZ(credentials=xyz_token)

In [ ]:
title = "My Demo Space"
description = "Demo"
space = xyz.spaces.new(title=title, description=description)
space_id = space.info["id"]

In [ ]:
# create map widget in sidebar (JupyterLab needed)
m = Map(zoom=2)
with Sidecar(title='XYZ'):
    display(m)

In [ ]:
# load https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json
# with minor clean-ups
gj = get_countries_data()

In [ ]:
# add world GeoJSON to XYZ space (returns the added GeoJSON)

world = space.add_features(features=gj)

In [ ]:
# add entire GeoJSON feature from XYZ to the map
m += GeoJSON(data=world)

In [ ]:
# add "full screen" button to map
m += FullScreenControl()

In [ ]:
fra = space.get_feature(feature_id='FRA')

In [ ]:
# add the feature to the map
style = dict(
    color="red",
    opacity=1,
    weight=1.9, 
    fillOpacity=0.5
)
m += GeoJSON(data=fra, style=style)

In [ ]:
# show only countries on southern hemisphere 

from geojson import FeatureCollection

bbox = [-180, -90, 180, 0]
bfeatures = []
for feature in space.features_in_bbox(bbox=[0, 0, 20, 20]):
    bfeatures.append(feature)
hemi = FeatureCollection(bfeatures)
m += GeoJSON(data=hemi, style=style)

In [ ]:
# show the same but with clipped borders 
for feature in space.features_in_bbox(bbox=[0, 0, 20, 20], clip=True):
    bfeatures.append(feature)
hemi = FeatureCollection(bfeatures)
style["color"] = "yellow"
m += GeoJSON(data=hemi, style=style)

In [ ]:
# Spatial search: Given latitude and longitude search intersecting geometry.

fc = []
for feature in space.spatial_search(lat=19.1526, lon=72.8557, radius=100000):
    fc.append(feature)
    
spatial_search = FeatureCollection(fc)
style["color"] = "orange"
m += GeoJSON(data=spatial_search, style=style)
m

In [ ]:
# clean-up and delete temporary space
space.delete()